In [ ]:
import os
import sys
# sys.path.append('../../')
sys.path.append(rf"C:\Git\BI0730")
from libs.geral.utils import * 
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
user = os.getlogin()
# from inner_libs.libs_doug import * 
# from libs.doug.utils_doug import * 

In [3]:
import duckdb

# Conectar ao DuckDB (ou criar um novo banco de dados em memória)
con = duckdb.connect()

# Ler arquivos Parquet de uma pasta
# Substitua 'caminho/para/sua/pasta/*.parquet' pelo caminho real
path = rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\\2025*.parquet"
query = rf'''
    CREATE TABLE dados AS
    SELECT * FROM read_parquet('{path}')
    
   
'''

con.execute(query)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:

query = f''' 
      WITH tratados AS 
        (
            SELECT *,
            ROW_NUMBER() OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes DESC) AS rn,
            pontos_principalidade - LAG(pontos_principalidade)  OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC) AS var_pontos

            FROM dados
            QUALIFY rn <= 4
            ORDER BY ano_Mes
        ),
        
        marcacoes AS (
            SELECT  
            *,
            CASE WHEN var_pontos < 0 THEN 1 ELSE 0 END AS queda_flag
            FROM tratados 
            WHERE rn <= 3
            
        ),
        checagem AS (
            SELECT *,
            SUM(queda_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_quedas
            FROM marcacoes
        )
        
        SELECT  * FROM checagem WHERE soma_quedas >= 3 and ano_mes = (SELECT MAX(ano_mes) FROM checagem)
        --SELECT  * FROM checagem WHERE cpf_cnpj = '00362741948'
        

        '''
con.sql(query).to_df().to_parquet()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,idade,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_agro_sicredi,total_valor_agro_mercado_scr,total_valor_outros_sicredi,total_valor_agro_mercado_cpr,sow_agro,total_valor_outros_mercado_scr,sow_outros,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_consorcio_motos,possui_adquirencia,possui_consorcio_pesados,possui_consorcio_servicos,possui_consorcio_automoveis,possui_consorcio_imoveis,possui_seguro_rural,possui_previdencia,possui_seguro_residencial,possui_seguro_agricola,possui_seguro_patrimonial,possui_seguro_vida,possui_seguro_automovel,possui_conta_salario,transacao_app,flg_ativo,flg_novo_assoc,flg_idade_maior_18,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,pontos_rural,pontos_planejamento,pontos_protecao,pontos_previdencia,pontos_movimentacao,pontos_cred_cartao,pontos_cred_outros,pontos_investimento,pontos_cred_rural,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,dat_carga_incremental,faixa_categoria,etl_created_on,etl_updated_on,data_insercao,data_atualizacao,filtro_ano_mes_inicial,filtro_ano_mes_final,rn,var_pontos,queda_flag,soma_quedas
0,202506,00362741948,PF,0730,7000,31,010101,ATIVO,PF I,46,202108,MÉDIO 2,DIGITAL,0.00,51,S,S,0.00,S,0.00,0.00,"2,110.57","1,719.22",0.00,0.00,288.41,0.00,0.00,"1,052.37",0.27,0.81,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,"41,459.11","41,016.02",0.00,"82,475.13",15,0,0,0,0,40,16,5,0,0,1,76,120+,152.0,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-5,1,3.00
1,202506,00490494960,PF,0730,7000,48,010101,ATIVO,PF I,8,202410,MÉDIO 1,DIGITAL,0.00,46,N,S,0.00,N,0.00,0.00,"2,077.25","2,077.25",0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.00,0.00,0.00,0.00,5,0,3,0,0,0,20,0,0,0,0,28,20-80%,56.00000000000001,2025-07-17,Emergente,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-13,1,3.00
2,202506,00601010914,PF,0730,7000,18,141,ATIVO,PF IV,111,201603,BAIXO 1,LEGADO,"3,683.77",41,S,S,"10,277.90",S,"24,821.38","13,961.67","79,428.67","22,399.04",0.00,0.00,"113,726.23",0.00,0.00,"195,287.42",0.58,0.28,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,"33,817.15","35,579.26",0.00,"69,396.41",16,0,0,0,12,100,14,29,6,0,1,177,120+,136.15384615384616,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-3,1,3.00
3,202506,00651418909,PF,0730,7000,28,111,ATIVO,PF I,73,201905,BAIXÍSSIMO,LEGADO,"1,821.23",48,S,S,"10,348.73",S,0.00,"12,169.96","14,339.72","6,679.32",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.47,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,"17,140.22","24,722.69",0.00,"41,862.91",15,0,0,3,0,40,9,0,40,0,1,107,120+,214.0,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-50,1,3.00
4,202506,00674134982,PF,0730,7000,04,111,ATIVO,PF II,33,202209,BAIXÍSSIMO,LEGADO,645.59,48,S,S,"3,500.00",S,0.00,"4,145.59","1,474.18","1,474.18",0.00,0.00,0.00,0.00,0.00,"1,826.51",0.00,1.00,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,800.00,"1,494.75",0.00,"2,294.75",20,0,0,0,0,11,30,0,20,0,1,81,100-120%,101.25,2025-07-17,Principal,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-1,1,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:

path_to = criar_pasta_exportacao("Alexandre Hekermann H - leads_visao")
salvar_jupyter_em_pasta(path_to, file_origin="Principal.ipynb", arquivo_reutilizado=False)
base_exportar.to_excel(os.path.join(path_to,"base_pesquisas.xlsx"),index=False)
# total.to_parquet(os.path.join(path_to,"base_total_raw.parquet"),index=False)

# with pd.ExcelWriter(os.path.join(path_to,"lotes_agroleite_2025.xlsx")) as writer:
#     agro_pf_tel.to_excel(writer, sheet_name="Agroleite PF", index=False)
#     agro_pj_tel.to_excel(writer, sheet_name="Agroleite PJ", index=False)

os.startfile(path_to)


['ano_mes', 'cpf_cnpj', 'segmento', 'cod_cooperativa', 'cod_central',
       'cod_agencia', 'cod_carteira', 'status_associado', 'porte_padrao',
       'meses_desde_associacao', 'assoc_desde', 'nivel_risco', 'des_origem',
       'vlr_capital_social',  'pix_trans_30d', 'cad_pix',
       'sum_titulo_investimento', 'mobi_transacionou_30d', 'sum_previdencia',
       'sum_investimentos', 'total_valor_cartao_mercado_scr',
       'total_valor_cartao_sicredi', 'total_valor_outros_sicredi',  
         'sow_cartao',
       'possui_cartao_credito', 'debito_conta_ativo', 'possui_domicilio',
       'possui_cartao_debito', 'possui_folha_pagamento', 'possui_cobranca',
       'possui_adquirencia'(maquineta),
         'transacao_app',
       'flg_ativo', 'flg_novo_assoc',  'cad_pix_ativo',
       'ativou_open_finance', 'cash_in', 'cash_out', 'cash_total_fator',
       'cash_total', 'pontos_produtos_basicos',  'flag_principalidade',
       'pontos_principalidade', 'faixa_atingimento_meta',
       'percentual_atingimento',  'faixa_categoria',
        'data_atualizacao',  'var_pontos',
       'queda_flag', 'soma_quedas']